In [2]:

# imports
import sys
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xarray as xr
import sys
import pickle

# include code path
sys.path.append('code')

# load custom shield scripts
from shield_utils import find_animals, get_lfp_dict, downsample, align_lfp, load_animals_oi

# paths
data_path = '/oscar/data/sjones/kduecker/shield_data'
meta_path = 'externals/SHIELD_Dynamic_Gating_Analysis'


#for subj_id in range(6):
# # subj_id from array job
# subj_id = 1

subj_id = 1
down_srate = 500            # downsampling
roi = ['LGd', 'VISp']       # regions of interest
toi = [0, 2]                # time window around 

mice_sess = load_animals_oi()  # load subject and sesson IDs

# loop over mice here and store
subj = list(mice_sess.keys())[subj_id]

ses_files = os.listdir(os.path.join(data_path,f'sub-{subj}'))           # sessions per mouse

# get lfp files and spike files
lfp_files = list(filter(lambda s: 'None' in s, ses_files))

# load the sessions that have the ROIs

session = mice_sess[subj][0]
ses_file = list(filter(lambda s: session in s, ses_files))
 
#layer_lfp = get_lfp_dict(subj, data_path, lfp_files, ses_file[0], toi, down_srate, roi) # empty...

# layer_area_units is empty!

In [2]:
# check if channel indices for single units and lfp are different (it looks like they are).

# sessions of interest

lfp_chan = dict()
# extract layer for each unit
layer_areas_units = dict()

single_units = dict()

for session in mice_sess[subj]:

    # fill in layer areas
    layer_areas_units[session] = dict()
    lfp_chan[session] = []
    single_units[session] = []
    ses_file = list(filter(lambda s: session in s, ses_files))
    nwb_file_asset = pynwb.NWBHDF5IO(f'{data_path}/sub-{subj}/{ses_file[0]}', mode='r', load_namespaces=True)
    nwb_file = nwb_file_asset.read()
    dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)

    # probe map
    probe_index = dynamic_gating_session.probes.index
    probe_map = {}
    for p in probe_index:
        probe_name = dynamic_gating_session.probes.name[p]
        filename = list(filter(lambda s: '-'+str(p)+'_' in s, lfp_files))
        probe_map[probe_name] = os.path.join(os.path.join(data_path,f'sub-{subj}'),filename[0])

    # add the LFP data to the session object
    dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file, probe_data_path_map=probe_map)

    # get the channels
    sess_units = dynamic_gating_session.get_units()
    single_units[session] = sess_units.index.values

    # find the different layers in the brain area (e.g. VISpl2/3. VISl4, VISpl5)

    areas = np.unique(sess_units.structure_layer.values)
    area_layers = [name for name in areas if any(r in name for r in roi)]

    for al in area_layers: 
        layer_areas_units[session][al] = sess_units[sess_units.structure_layer.str.contains(al)].index

    # get stimulus presentations
    stim_presentations = dynamic_gating_session.stimulus_presentations
    flashes = stim_presentations[stim_presentations['stimulus_name'].str.contains('flash')]
    presentation_times = flashes.start_time.values
    flash_end_times = presentation_times + flashes.duration
    presentation_ids = flashes.index.values

    srate = dynamic_gating_session.probes.sampling_rate.values[0]
    dt = 1/srate


    # load LFP for each probe
    for pi in probe_index:
        probe_lfp = dynamic_gating_session.get_lfp(pi)
        lfp_chan[session].append(probe_lfp.channel.values)

 

In [9]:
for session in lfp_chan:
    lu = np.concatenate(lfp_chan[session])
    su = single_units[session]
    if len(np.intersect1d(lu,su)) == 0:
        print(f'session {session} no overlap')

no overlap
no overlap


In [6]:
# all sessions

lfp_chan = dict()
# extract layer for each unit
layer_areas_units = dict()

single_units = dict()

for ses_file in ses_files[:4]:
    session = ses_file[ses_file.find('ses-'):ses_file.find('T')+1]
    layer_areas_units[session] = dict()
    lfp_chan[session] = []
    single_units[session] = []
    #ses_file = list(filter(lambda s: session in s, ses_files))
    nwb_file_asset = pynwb.NWBHDF5IO(f'{data_path}/sub-{subj}/{ses_file}', mode='r', load_namespaces=True)
    nwb_file = nwb_file_asset.read()
    dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)

    # probe map
    probe_index = dynamic_gating_session.probes.index
    probe_map = {}
    for p in probe_index:
        probe_name = dynamic_gating_session.probes.name[p]
        filename = list(filter(lambda s: '-'+str(p)+'_' in s, lfp_files))
        probe_map[probe_name] = os.path.join(os.path.join(data_path,f'sub-{subj}'),filename[0])

    # add the LFP data to the session object
    dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file, probe_data_path_map=probe_map)

    # get the channels
    sess_units = dynamic_gating_session.get_units()
    single_units[session].append(sess_units.index.values)

    # find the different layers in the brain area (e.g. VISpl2/3. VISl4, VISpl5)

    areas = np.unique(sess_units.structure_layer.values)
    area_layers = [name for name in areas if any(r in name for r in roi)]

    for al in area_layers: 
        layer_areas_units[session][al] = sess_units[sess_units.structure_layer.str.contains(al)].index

    # get stimulus presentations
    stim_presentations = dynamic_gating_session.stimulus_presentations
    flashes = stim_presentations[stim_presentations['stimulus_name'].str.contains('flash')]
    presentation_times = flashes.start_time.values
    flash_end_times = presentation_times + flashes.duration
    presentation_ids = flashes.index.values

    srate = dynamic_gating_session.probes.sampling_rate.values[0]
    dt = 1/srate


    # load LFP for each probe
    for pi in probe_index:
        probe_lfp = dynamic_gating_session.get_lfp(pi)
        lfp_chan[session].append(probe_lfp.channel.values)

In [7]:
for session in lfp_chan:
    lu = np.concatenate(lfp_chan[session])
    su = single_units[session]
    if len(np.intersect1d(lu,su)) == 0:
        print(f'session {session} no overlap')
    else:
        print(f'session {session} overlap')

session ses-20220926T no overlap
session ses-20220927T no overlap
session ses-20220928T no overlap
session ses-20220929T no overlap


In [5]:
ses_file

'sub-626279_ses-None_probe-355_ecephys.nwb'

In [ ]:
ses_file[]